In [ ]:
# Requirements for this notebook to run: pandas, geopandas, shapely, pyproj
import concurrent.futures
import os
import time
import json

import pandas
import geopandas
from shapely.geometry import shape
import contextily as ctx

from eocalc.context import Pollutant
from eocalc.methods.base import DateRange
from eocalc.methods.temis import TEMISTropomiMonthlyMeanNOxEmissionCalculator

In [ ]:
directory = "data/regions"
regions = {}

for filename in os.listdir(directory):
    if filename.endswith(".geo.json") or filename.endswith(".geojson"):
        with (open(f"{directory}/{filename}", 'r')) as geojson_file:
            regions[filename] = shape(json.load(geojson_file)["geometry"])

In [ ]:
results = {} # results will be put here as results[<filename>][<data>]

def run_method(filename, region):
    results[filename] = TEMISTropomiMonthlyMeanNOxEmissionCalculator().run(region, DateRange(start='2018-08-01', end='2018-08-31'), Pollutant.NOx)
    print(f"Done with region represented by file '{filename}'")

with concurrent.futures.ThreadPoolExecutor() as executor:
    for filename, region in regions.items():
        executor.submit(run_method, filename, region)

In [ ]:
for filename in regions:
    print(f"Total emissions in region {filename}: {results[filename][TEMISTropomiMonthlyMeanNOxEmissionCalculator.TOTAL_EMISSIONS_KEY]/10**6:.2f}kt")

In [ ]:
for filename in regions:
    gridded_result = results[filename][TEMISTropomiMonthlyMeanNOxEmissionCalculator.GRIDDED_EMISSIONS_KEY]
    gridded_result.plot("total emissions [kg]", figsize=(20, 20), legend=True, legend_kwds={'label': f"Emissions in {filename} [kg]", 'orientation': "horizontal"})

In [ ]:
for filename in regions:
    gridded_result = results[filename][TEMISTropomiMonthlyMeanNOxEmissionCalculator.GRIDDED_EMISSIONS_KEY]
    ctx.add_basemap(gridded_result.to_crs(epsg=3857).plot("total emissions [kg]", figsize=(20, 20), alpha=0.7, edgecolor='k'))